In [1]:
# https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/main/22.%20Fine-tuning%20GPT-2%20(Cls%2C%20Chatbot%2C%20NLI)/22-5.%20kogpt2_kornli_tpu.ipynb

# pip install transformers

# title 추가
# https://github.com/hayoon/nlp_hate_speech/blob/01b371291269f9fdcf31010e24759c78e828fa8b/code/yeji/01_data_skimming(2).ipynb#L52

In [2]:
import transformers
transformers.__version__

'4.35.2'

In [3]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn import preprocessing
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
import sys

print(os.getcwd())

/content


In [6]:
!pwd
!ls

/content
drive  sample_data


In [7]:
import pandas as pd
train_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/train.hate.csv')
dev_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/dev.hate.csv')
test_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/test.hate.no_label.csv')

# , sep='\t', quoting=3

In [8]:
train_newstitle = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/train.news_title.txt', sep='\t', names=['news_title'])

# train_newstitle.rename(columns={'label': 'title_label'}, inplace=True)
train_newstitle.tail()

,news_title
7891,"허지웅, 허투루 넘길 말 없었다…솔직하게 드러냈던 속사정"
7892,"이혜경, ‘오! 캐롤’ 공연 중 남편 오정욱 부고…오열 속 발인 [종합]"
7893,"'설경구♥' 송윤아, 아들과 즐거운 하루 ""전 엄마니까요"""
7894,"[SC현장]""연예인 인생 협박 유감""…미소잃은 최민수, '보복운전 혐의' 2차 공판..."
7895,"[단독]스태프 사망사고 '서른이지만', 결국 오늘 촬영 취소"


In [9]:
test_data.tail()

,comments
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...
972,입에 손가릭이 10개 있으니 징그럽다
973,난 조보아 이뻐서 보는데 백종원 관심무


In [10]:
from functools import reduce

df = [train_data, train_newstitle]

train = reduce(lambda left, right: pd.merge(left, right, on = train_data['comments']), df)
train = train.drop('key_0', axis=1)
train.tail()

,comments,label,news_title
7891,힘내세요~ 응원합니다!!,none,"허지웅, 허투루 넘길 말 없었다…솔직하게 드러냈던 속사정"
7892,힘내세요~~삼가 고인의 명복을 빕니다..,none,"이혜경, ‘오! 캐롤’ 공연 중 남편 오정욱 부고…오열 속 발인 [종합]"
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,none,"'설경구♥' 송윤아, 아들과 즐거운 하루 ""전 엄마니까요"""
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,none,"[SC현장]""연예인 인생 협박 유감""…미소잃은 최민수, '보복운전 혐의' 2차 공판..."
7895,힘들면 관뒀어야지 그게 현명한거다,none,"[단독]스태프 사망사고 '서른이지만', 결국 오늘 촬영 취소"


In [11]:
is_NaN = train.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = train[row_has_NaN]
rows_with_NaN

,comments,label,news_title
1522,남의 일에는 신경안써' 하고 쿨한척 하던 인간들. 강남한테는 악플 쓰고 있음ㅋㅋㅋ ...,hate,NaN
5574,"응애 응애 엄마 저 맘에 안들죠? ........아들 ?? "" 너 내가 우스워 보이...",hate,NaN
7150,초콜릿' 드럽게 재미없다 ㅠㅠ 다들 공감하실듯,offensive,NaN


In [12]:
train['news_title'].fillna("No Title", inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7896 entries, 0 to 7895
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   comments    7896 non-null   object
 1   label       7896 non-null   object
 2   news_title  7896 non-null   object
dtypes: object(3)
memory usage: 246.8+ KB


In [13]:
# # 훈련 데이터 다운로드
# urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
# urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")

# # 검증 데이터 다운로드
# urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")

# # 테스트 데이터 다운로드
# urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")

In [14]:
# train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
# train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
# val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
# test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

In [15]:
train.head()

,comments,label,news_title
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate,"밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]"
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none,"'연중' 故 전미선, 생전 마지막 미공개 인터뷰…환하게 웃는 모습 '먹먹'[종합]"
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate,"[단독] 잔나비, 라디오 출연 취소→'한밤' 방송 연기..비판 여론 ing(종합)"
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none,"'아스달 연대기' 장동건-김옥빈, 들끓는 '욕망커플'→눈물범벅 '칼끝 대립'"
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate,[DA:이슈] ‘구하라 비보’ 최종범 항소심에 영향?…법조계 “‘공소권 없음’ 아냐”


In [16]:
dev_data['label']

0           none
1      offensive
2           hate
3           hate
4      offensive
         ...    
466    offensive
467         hate
468    offensive
469         hate
470         none
Name: label, Length: 471, dtype: object

In [17]:
# text to index mapping

LABEL_DIC = {
    'none': 0,
    'offensive': 1,
    'hate': 2,
}

train['label'] = train['label'].map(lambda x: LABEL_DIC[x])
dev_data['label'] = dev_data['label'].map(lambda x: LABEL_DIC[x])

## 데이터 정제

In [18]:
import string

## remove basic punctuation
def remove_punc(text):
    text_nopunc = "".join([char for char in text if char not in string.punctuation])
    return text_nopunc

train['comments'] = train['comments'].apply(lambda x: remove_punc(x))
train['news_title'] = train['news_title'].apply(lambda x: remove_punc(x))

dev_data['comments'] = dev_data['comments'].apply(lambda x: remove_punc(x))

test_data['comments'] = test_data['comments'].apply(lambda x: remove_punc(x))

In [19]:
import re

# remove all punctuations except korean, english, and number
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[a-zA-Z]', '', text)
    text = re.sub('[ㄱ-ㅎㅏ-ㅣ]+', '', text)
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)
    return text

train['comments'] = train['comments'].apply(cleanse)
train['news_title'] = train['news_title'].apply(cleanse)

dev_data['comments'] = dev_data['comments'].apply(cleanse)

test_data['comments'] = test_data['comments'].apply(cleanse)

train.head()

,comments,label,news_title
0,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네,2,밤새 조문 행렬故 전미선 동료들이 그리워하는 따뜻한 배우 종합
1,한국적인 미인의 대표적인 분너무나 곱고아름다운모습그모습뒤의 슬픔을 미처 알지못했네요,0,연중 故 전미선 생전 마지막 미공개 인터뷰환하게 웃는 모습 먹먹종합
2,못된 넘들남의 고통을 즐겼던 넘들이젠 마땅한 처벌을 받아야지그래야 공정한 사회지심은...,2,단독 잔나비 라디오 출연 취소→한밤 방송 연기비판 여론 종합
3,12화 어설펐는데 34화 지나서부터는 갈수록 너무 재밌던데,0,아스달 연대기 장동건김옥빈 들끓는 욕망커플→눈물범벅 칼끝 대립
4,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노,2,이슈 구하라 비보’ 최종범 항소심에 영향법조계 “공소권 없음’ 아냐”


In [20]:
train['comments'] = train['comments'].str.replace(' ', '')
train['news_title'] = train['news_title'].str.replace(' ', '')

train['comments'] = train['comments'] + train['news_title']

train['comments']

0       현재호텔주인심정아18난마른하늘에날벼락맞고호텔망하게생겼는데누군계속추모받네밤새조문행렬故...
1       한국적인미인의대표적인분너무나곱고아름다운모습그모습뒤의슬픔을미처알지못했네요연중故전미선생...
2       못된넘들남의고통을즐겼던넘들이젠마땅한처벌을받아야지그래야공정한사회지심은대로거두거라단독잔...
3       12화어설펐는데34화지나서부터는갈수록너무재밌던데아스달연대기장동건김옥빈들끓는욕망커플→...
4       1사람얼굴손톱으로긁은것은인격살해이고2동영상이몰카냐메걸리안들생각이없노이슈구하라비보’최...
                              ...                        
7891                     힘내세요응원합니다허지웅허투루넘길말없었다솔직하게드러냈던속사정
7892              힘내세요삼가고인의명복을빕니다이혜경오캐롤’공연중남편오정욱부고오열속발인종합
7893                     힘내세용항상응원합니닷설경구♥송윤아아들과즐거운하루전엄마니까요
7894    힘내소연기로답해요나도53살인데이런일저런일다있더라구요인격을믿습니다홨팅현장연예인인생협박...
7895                힘들면관뒀어야지그게현명한거다단독스태프사망사고서른이지만결국오늘촬영취소
Name: comments, Length: 7896, dtype: object

In [21]:
# 필요 없는 코드!
# train_data_list = []

# for q, p in zip(train['comments'], train['news_title'])  :
#     data = []
#     data.append(q + p)
#     # data.append(p)

#     train_data_list.append(data)

# # 뉴스타이틀제거버전

# # train_data_list = []

# # for q, label in zip(train['comments'], train['label'])  :
# #     data = []
# #     data.append(q)
# #     # data.append(p)
# #     data.append(str(label))

# #     train_data_list.append(data)

In [22]:
# dev_data_list = []

# for q, label in zip(dev_data['comments'], dev_data['label'])  :
#     data = []
#     data.append(q)
#     data.append(str(label))

#     dev_data_list.append(data)

In [23]:
# train_data_list
# dev_data_list

In [24]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')

In [25]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))
tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

for elem in tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"):
  print(tokenizer.decode(elem))

print(tokenizer.tokenize("전율을 일으키는 영화. 다시 보고싶은 영화"))

print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))

for elem in tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"):
  print(tokenizer.decode(elem))

for elem in tokenizer.encode("happy birthday~!"):
  print(tokenizer.decode(elem))

print(tokenizer.decode(0))
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

[11867, 7071, 7071, 10554, 9359, 7498, 7162, 15305, 9488, 10191, 7487, 9712, 9868, 8031]
['▁보는', '내', '내', '▁그대로', '▁들어', '맞', '는', '▁예측', '▁카', '리스', '마', '▁없는', '▁악', '역']
보는
내
내
그대로
들어
맞
는
예측
카
리스
마
없는
악
역
['▁전', '율을', '▁일으키는', '▁영화', '.', '▁다시', '▁보고', '싶', '은', '▁영화']
[9034, 13555, 16447, 10584, 389, 9427, 10056, 7898, 8135, 10584]
전
율을
일으키는
영화
.
다시
보고
싶
은
영화

ha
pp
y
b
ir
th
d
ay
~
!
<s>
</s>
<usr>
<pad>
<sys>


In [26]:
max_seq_len = 128

In [27]:
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=max_seq_len, padding='max_length')
print(encoded_result)
print('길이 :', len(encoded_result))

print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=128))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[9034, 13555, 16447, 10584, 389, 9427, 10056, 7898, 8135, 10584, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
길이 : 128
[9034, 13555, 16447, 10584, 389, 9427, 10056, 7898, 8135, 10584]


In [28]:
def convert_examples_to_features(comments, max_seq_len, tokenizer):

    input_ids = []

    for sent1 in tqdm(comments, total = len(comments)):
        # bos_token = [tokenizer.bos_token]
        # eos_token = [tokenizer.eos_token]
        # sent1_tokens = bos_token + tokenizer.tokenize(sent1) + eos_token + ['']

        sent1_tokens = [tokenizer.bos_token] + tokenizer.tokenize(sent1) + [tokenizer.eos_token]

        tokens = sent1_tokens
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)

    input_ids = np.array(input_ids, dtype=int)

    return input_ids

In [29]:
X_train = convert_examples_to_features(train['comments'], max_seq_len = max_seq_len, tokenizer = tokenizer)

100%|██████████| 7896/7896 [00:06<00:00, 1202.07it/s]


In [30]:
# 최대 길이: 128
input_id = X_train[0]

print('단어에 대한 정수 인코딩 :', input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    1  9490 31569 17653  7894  8194  7965 11050  7058  7487  7469 22296
  8022  7060  7641  7373  7498  6889 31569  7497  9124  7777  6883  9668
  7139  6921 28905  8397  7540  7600  7098  7604  7769  8213  7559  8719
  7418  2786  8187  7584  7791  7244  7442  9136 22506 40141  7303  7359
  8704 20268 17207     1     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 128
정수 인코딩 복원 : </s> 현재호텔주인심정아18난마른하늘에날벼락맞고호텔망하게생겼는데누군계속추모받네밤새조문행렬故전미선동료들이그리워하는따뜻한배우종합</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [31]:
dev_data['comments']

0                              송중기 시대극은 믿고본다 첫회 신선하고 좋았다
1                                                지현우 나쁜놈
2             알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라
3                                          설마  현정 작가 아니지
4      이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...
                             ...                        
466                                     지현우 범죄 저지르지 않았나요
467                                        여자인생 망칠 일 있나 
468               근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는
469       할매젖 뱃살 몸매 라인 유륜은 적당해야됨너무크거나 너무 작아도  엉덩이가 빵빵해야됨
470    남자가 잘못한거라면 반성도 없다면나였다면 여자처럼 아주 못되게 할것같다왜 나를 배신...
Name: comments, Length: 471, dtype: object

In [32]:
X_dev = convert_examples_to_features(dev_data['comments'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 471/471 [00:00<00:00, 1312.69it/s]


In [33]:
# 최대 길이: 128
input_id = X_dev[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    1  9697  8243  6958 10343 29960 18882  7655  7182  9877  8765 16035
  9038  9677  7978  7182     1     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 128
정수 인코딩 복원 : </s> 송중기 시대극은 믿고본다 첫회 신선하고 좋았다</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [34]:
X_test = convert_examples_to_features(test_data['comments'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 974/974 [00:00<00:00, 1196.14it/s]


In [35]:
train_label = train['label'].tolist()
dev_label = dev_data['label'].tolist()
# test_label = test_data['label'].tolist() # test는 라벨 없음

dev_data['label']
train['label']

0       2
1       0
2       2
3       0
4       2
       ..
7891    0
7892    0
7893    0
7894    0
7895    0
Name: label, Length: 7896, dtype: int64

In [36]:
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

y_train = idx_encode.transform(train_label) # 주어진 고유한 정수로 변환
y_dev = idx_encode.transform(dev_label) # 고유한 정수로 변환
# y_test = idx_encode.transform(test_label) # 고유한 정수로 변환 # test는 라벨 없음

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print(label_idx)
print(idx_label)

{0: 0, 1: 1, 2: 2}
{0: 0, 1: 1, 2: 2}


In [37]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [38]:
max_seq_len = 128

In [39]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [40]:
print(outputs)
print(outputs[0])
print(outputs[1])
print(outputs[0][:, -1])

TFBaseModelOutputWithPastAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tfgpt2_model')>, past_key_values=(<KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')

In [41]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        prediction = self.classifier(cls_token)

        return prediction

TPU 사용법 : https://wikidocs.net/119990

In [42]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [43]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [44]:
with strategy.scope():
  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=3)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [45]:
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta = 0.001,
    patience = 2)

# epoch을 1만 돌리기 ?
model.fit(
    X_train, y_train, epochs=10, batch_size=64, validation_data=(X_dev, y_dev),
    callbacks = [early_stopping]
)

Epoch 1/10
124/124 [==============================] - 90s 142ms/step - loss: 1.4094 - accuracy: 0.4022 - val_loss: 1.2723 - val_accuracy: 0.3397
Epoch 2/10
124/124 [==============================] - 13s 105ms/step - loss: 1.0377 - accuracy: 0.4757 - val_loss: 1.2551 - val_accuracy: 0.3779
Epoch 3/10
124/124 [==============================] - 13s 107ms/step - loss: 0.8671 - accuracy: 0.5813 - val_loss: 1.1080 - val_accuracy: 0.4459
Epoch 4/10
124/124 [==============================] - 13s 105ms/step - loss: 0.7456 - accuracy: 0.6583 - val_loss: 1.0191 - val_accuracy: 0.4968
Epoch 5/10
124/124 [==============================] - 13s 108ms/step - loss: 0.6204 - accuracy: 0.7290 - val_loss: 1.4011 - val_accuracy: 0.4671
Epoch 6/10
124/124 [==============================] - 13s 106ms/step - loss: 0.5288 - accuracy: 0.7696 - val_loss: 1.1787 - val_accuracy: 0.5074
Epoch 7/10
124/124 [==============================] - 13s 105ms/step - loss: 0.4016 - accuracy: 0.8350 - val_loss: 1.2015 - val_ac

In [46]:
# 토큰화 된 상태
# X_test

X_test

array([[    1, 31416,  9059, ...,     3,     3,     3],
       [    1,  9710,  7182, ...,     3,     3,     3],
       [    1,  9252,  7220, ...,     3,     3,     3],
       ...,
       [    1, 13163,  9446, ...,     3,     3,     3],
       [    1, 29521,  9526, ...,     3,     3,     3],
       [    1,  9769,  9059, ...,     3,     3,     3]])

In [47]:
output = model.predict(X_test)

31/31 [==============================] - 6s 48ms/step


In [48]:
output

array([[9.0514666e-01, 7.8518093e-02, 1.6335255e-02],
       [9.9901450e-01, 9.4510568e-04, 4.0347179e-05],
       [9.4349623e-01, 5.5865057e-02, 6.3878595e-04],
       ...,
       [9.9916172e-01, 8.2869554e-04, 9.6321510e-06],
       [9.2038745e-01, 7.7459022e-02, 2.1535540e-03],
       [9.9971944e-01, 2.7022720e-04, 1.0408610e-05]], dtype=float32)

In [49]:
import torch

outputTensor = torch.Tensor(output)

In [50]:
# outputTensor = nn.Softmax(dim=1)(outputTensor)
pred = torch.argmax(outputTensor, dim=-1)

pred

tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 2, 0, 0, 2, 0, 0,
        0, 1, 2, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 2, 0, 1, 1, 0, 0, 0, 1, 1,
        2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
        1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 2, 1, 1, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 2, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,

In [51]:
test_label = [int(k.cpu()) for k in pred]
test_data['label'] = test_label

In [52]:
test_data.to_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/submission_kogptCleanVer1_resetALL.csv', index=None)